In [1]:
# TEMPERATURE RANGE

import arcpy
import csv
Days = ["05/01", "15/01", "25/01", "05/02", "15/02", "25/02", "05/03", "15/03", "25/03", "05/04", "15/04", "25/04", \
         "05/05", "15/05", "25/05", "05/06", "15/06", "25/06", "05/07", "15/07", "25/07", "05/08", "15/08", "25/08", \
         "05/09", "15/09", "25/09", "05/10", "15/10", "25/10", "05/11", "15/11", "25/11", "05/12", "15/12", "25/12"]
FolderBase = r"C:\\DataDanny\\Agrichange\\Climate data\\Agriclimatic indicators Copernicus\\"
FolderGrowthSeason = FolderBase + "Mean of daily mean temperature\\"
FolderOut = r"C:\\DataDanny\\Agrichange\\Climate data\\TemporaryOutputs\\"

FolderA = FolderBase + "Maximum value of daily maximum temperature\\"
FolderB = FolderBase + "Minimum value of the daily minimum Temperature\\"

NCInfileA1 = FolderA + "TXx_C3S-glob-agric_hadgem2-es_hist_dek_19510101-19801231_v1.1.nc"
NCInfileA2 = FolderA + "TXx_C3S-glob-agric_hadgem2-es_hist_dek_19810101-20101231_v1.1.nc"
NCInfileA3 = FolderA + "TXx_C3S-glob-agric_hadgem2-es_rcp8p5_dek_20110101-20401231_v1.1.nc"
NCInfileB1 = FolderB + "TNn_C3S-glob-agric_hadgem2-es_hist_dek_19510101-19801231_v1.1.nc"
NCInfileB2 = FolderB + "TNn_C3S-glob-agric_hadgem2-es_hist_dek_19810101-20101231_v1.1.nc"
NCInfileB3 = FolderB + "TNn_C3S-glob-agric_hadgem2-es_rcp8p5_dek_20110101-20401231_v1.1.nc"
TypeStats = "MEAN"
NameIndicator = "RANGE"
VariableTypeA = "TXx"
VariableTypeB = "TNn"

InputLocations = "LTE-StartSet"
Start_Year = 1955
End_Year = 1965 # note add 1

with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        Message = "Running Year: " + str(year) + "           "
        print(Message, end="\r")
        OutFile1 = FolderOut + NameIndicator + "Temp_"+ str(year) + ".csv"
        OutFile2 = FolderOut + "Season" + NameIndicator + "Temp_"+ str(year) + ".csv"
        # Extract and Name Days per year
        if year < 1981:
            NCInfileA = NCInfileA1
            NCInfileB = NCInfileB1
        elif year > 2010:
            NCInfileA = NCInfileA3
            NCInfileB = NCInfileB3
        else:
            NCInfileA = NCInfileA2
            NCInfileB = NCInfileB2
        Count = 0
        for date in range(0,len(Days)):
            Count = Count + 1;
            Dimension = "time '"+ Days[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNametmpA = "TmpDatesA_" + str(Count)
            LayerNametmpB = "TmpDatesB_" + str(Count)
            LayerName = "Dates_" + str(Count)
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfileA, variable=VariableTypeA, x_dimension="lon", y_dimension="lat",
                   out_raster_layer=LayerNametmpA, band_dimension="", dimension_values=Dimension,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfileB, variable=VariableTypeB, x_dimension="lon", y_dimension="lat",
                   out_raster_layer=LayerNametmpB, band_dimension="", dimension_values=Dimension,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Minus(in_raster_or_constant1=LayerNametmpA,in_raster_or_constant2= LayerNametmpB)
            out_raster.save(LayerName) 
            if date == 0:
                LayersArrayFull = LayerName
            else:
                LayersArrayFull = LayersArrayFull + ";" + LayerName   
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFull,
                     statistics_type=TypeStats,ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save("MinimumYearFull") 
        TableName = "ZonalFull"
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="code",in_value_raster= "MinimumYearFull", out_table=TableName,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
        NewName = "Y_" + str(year)
        arcpy.management.AlterField(in_table=TableName,field="MEAN",new_field_name=NewName,new_field_alias=NewName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR") 
        # Extract per LTE
        InFile1 = FolderGrowthSeason + "FirstDate_"+ str(year) + ".tif"
        InFile2 =FolderGrowthSeason + "LastDate_"+ str(year) + ".tif"
        LTECount = 0;
        arcpy.Delete_management("Zonalrast")     
        arcpy.Delete_management("Zonalrast2") 
        Start_Seq = 0
        expression=" SeqNum >= "+ str(Start_Seq)
        #expression=" SeqNum < 12"
        with arcpy.da.SearchCursor(InputLocations, ["code"], where_clause=expression) as cursor: 
            for row in cursor:          
                Message = "Running LTE: " + (row[0]) + " for " + str(year) + "           "
                print(Message, end="\r")
                exp = "code = '" + (row[0]) + "'" 
                LTECount = LTECount + 1;
                # Determine Growth season
                arcpy.analysis.Select(in_features=InputLocations,out_feature_class="TestedLTE", where_clause=exp)
                arcpy.sa.ZonalStatisticsAsTable(in_zone_data="TestedLTE", zone_field="code",in_value_raster= InFile1, out_table="Zonalrast",
                         ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                         percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
                SC = "Zonalrast"
                field_name = "Mean"
                values = [row[0] for row in arcpy.da.SearchCursor(SC, field_name)]
                Begintmp = values[0]
                Begin = arcpy.sa.Int(in_raster_or_constant= Begintmp)
                arcpy.sa.ZonalStatisticsAsTable(in_zone_data="TestedLTE",zone_field="code", in_value_raster=InFile2, out_table="Zonalrast2",
                            ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                            percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360) 
                SC = "Zonalrast2"
                field_name = "Mean"
                values = [row[0] for row in arcpy.da.SearchCursor(SC, field_name)]
                Endtmp = values[0]
                End = arcpy.sa.Int(in_raster_or_constant= Endtmp)
                # Growth season layers to be used as array
                Count = 0;              
                for date in range(Begin,End):
                    LayerName = "Dates_" + str(date)
                    if date == Begin:
                        LayersArraySeason = LayerName
                    else:
                        LayersArraySeason = LayersArraySeason + ";" + LayerName
                # Extract from Layers        
                if  LTECount == 1:
                    TableNameSeason = "ZonalSeason"
                else:
                    TableNameSeason = "TmpSeason"
                NewName = "LTE_" + row[0]
                # the cell statistics layer for full is calculated above already    
                out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArraySeason,
                             statistics_type=TypeStats,ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                             percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
                out_raster.save("MinimumYearSeason")    
                
                arcpy.sa.ZonalStatisticsAsTable(in_zone_data="TestedLTE", zone_field="code",in_value_raster= "MinimumYearSeason", out_table=TableNameSeason,
                             ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                             percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
                arcpy.management.AlterField(in_table=TableNameSeason,field="MEAN",new_field_name=NewName,new_field_alias=NewName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
                        
                if  LTECount != 1:
                    arcpy.management.JoinField(in_data="ZonalSeason",in_field="OBJECTID",join_table="TmpSeason",join_field="OBJECTID",
                         fields=NewName,fm_option="NOT_USE_FM",field_mapping=None)
        arcpy.management.DeleteField(in_table="ZonalSeason",drop_field="code;ZONE_CODE",method="DELETE_FIELDS")
        arcpy.management.AlterField(in_table="ZonalSeason",field="COUNT",new_field_name="Year",new_field_alias="YEAR",
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        arcpy.management.CalculateField(in_table="ZonalSeason",field="Year",expression=year,expression_type="PYTHON3",
                                        code_block="",field_type="DOUBLE",enforce_domains="NO_ENFORCE_DOMAINS")

        #end of per LTE loop 
        for i in range(0,2):
            if i == 0:
                table = "ZonalFull" 
                OutFile = OutFile1 
            else:
                table = "ZonalSeason" 
                OutFile = OutFile2    
            fields = arcpy.ListFields(table)  
            field_names = [field.name for field in fields]  
            with open(OutFile,'w') as f:  
                w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
                w.writerow(field_names)  
                for mow in arcpy.SearchCursor(table):  
                    field_vals = [mow.getValue(field.name) for field in fields]  
                    w.writerow(field_vals)                     

In [1]:
# population pressure

import arcpy
import csv

Years = [1975,1980,1985,1990,1995,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,\
         2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,20001,20051,20101,20151,20201]
FolderIn = r"C:\\DataDanny\\Agrichange\\Population data\\"
FolderOut = r"C:\\DataDanny\\Agrichange\\Climate data\\TemporaryOutputs\\"
InputLocations = "LTE-StartSet"
with arcpy.EnvManager(overwriteOutput=True):
    for year in range(30,len(Years)):
        InFile = FolderIn + "ppp_" + str(Years[year]) + "_1km_Aggregated.tif" 
        OutFile =  FolderOut + "PopulationPressure_" + str(Years[year]) + ".csv"
        LTECount = 0;
        Start_Seq = 0
        expression=" SeqNum >= "+ str(Start_Seq)
        with arcpy.da.SearchCursor(InputLocations, ["code"], where_clause=expression) as cursor: 
           for row in cursor:          
               Message = "Running LTE: " + (row[0]) + " for " + str(Years[year]) + "           "
               print(Message, end="\r")
               exp = "code = '" + (row[0]) + "'" 
               LTECount = LTECount + 1;

               arcpy.analysis.Select(in_features=InputLocations,out_feature_class="TestedLTE", where_clause=exp)
               arcpy.management.Project(in_dataset="TestedLTE",out_dataset="LTE_Project",
                        out_coor_system='PROJCS["WGS_1984_World_Mercator",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],UNIT["Meter",1.0]]',
                        transform_method=None,in_coor_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
                        preserve_shape="NO_PRESERVE_SHAPE",max_deviation=None,vertical="NO_VERTICAL")
               arcpy.analysis.Buffer(in_features="LTE_Project", out_feature_class="LTE_Buffer",buffer_distance_or_field="25000 Meters",line_side="FULL",
                                     line_end_type="ROUND",dissolve_option="NONE",dissolve_field=None,method="PLANAR") 
               ClipextentBuffered = arcpy.Describe("LTE_Buffer").extent
               Extent = str(ClipextentBuffered)   
               
               # This is the buffer clipped population layer
               arcpy.management.Clip(in_raster=InFile,rectangle=Extent,out_raster="InClipped",in_template_dataset="LTE_Buffer",
                     nodata_value="-999",clipping_geometry="ClippingGeometry",maintain_clipping_extent="MAINTAIN_EXTENT")
               arcpy.Delete_management("ProjectInRast") 
               arcpy.management.ProjectRaster(in_raster="InClipped",out_raster="ProjectInRast",
                       out_coor_system='PROJCS["WGS_1984_World_Mercator",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],UNIT["Meter",1.0]]',
                       resampling_type="NEAREST",cell_size="1000 1000",geographic_transform=None,Registration_Point=None,
                       in_coor_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
                       vertical="NO_VERTICAL")

                # Distance raster
               arcpy.Delete_management("WeightFraction") 
               arcpy.Delete_management("DistanceRast") 
               arcpy.Delete_management("KMRast") 
               arcpy.Delete_management("MinusfactorRast") 
               arcpy.Delete_management("ExpRastpreD2") 
               arcpy.Delete_management("Two_D_Pi_correction") 
               arcpy.Delete_management("ConRast") 
               out_raster = arcpy.sa.Con(in_conditional_raster="ProjectInRast",in_true_raster_or_constant=1,in_false_raster_or_constant=0,
                                          where_clause="VALUE >= -1")
               out_raster.save("ConRast")
               out_distance_raster = arcpy.sa.CostDistance(in_source_data="LTE_Project",in_cost_raster="ConRast",maximum_distance=None,
                       out_backlink_raster=None,source_cost_multiplier=None,source_start_cost=None,source_resistance_rate=None,
                       source_capacity=None,source_direction="")
               out_distance_raster.save("DistanceRast")
               out_raster = arcpy.sa.Divide(in_raster_or_constant1="DistanceRast",in_raster_or_constant2=1000)
               out_raster.save("KMRast")
               out_raster = arcpy.sa.Minus(in_raster_or_constant1=0,in_raster_or_constant2="KMRast")
               out_raster.save("MinusfactorRast")
               out_raster = arcpy.sa.Exp(in_raster_or_constant="MinusfactorRast")
               out_raster.save("ExpRastpreD2") 
               out_raster = arcpy.sa.Times(in_raster_or_constant1="KMRast",in_raster_or_constant2=6.28318530717959)
               out_raster.save("Two_D_Pi_correction")
               # This is the distance raster after applying a exponential curve and 2D correction that will be used as weighting on population
               out_raster = arcpy.sa.Divide(in_raster_or_constant1="ExpRastpreD2",in_raster_or_constant2="Two_D_Pi_correction")
               out_raster.save("WeightFraction") 
               
               arcpy.Delete_management("Deposition") 
                # the actual weighting 
               out_raster = arcpy.sa.Times(in_raster_or_constant1="WeightFraction",in_raster_or_constant2="ProjectInRast")
               out_raster.save("Deposition")
                
               # The extraction 
               TableLTEYear = "LTEYear"
               if LTECount == 1:
                   LTEtmpn = TableLTEYear
               else:
                   LTEtmpn = "TableTmp"
               NewName = "LTE_" + row[0]
               arcpy.ia.ZonalStatisticsAsTable(in_zone_data="LTE_Buffer",zone_field="OBJECTID",in_value_raster="Deposition",
                     out_table=LTEtmpn,ignore_nodata="DATA",statistics_type="SUM",process_as_multidimensional="CURRENT_SLICE",
                     percentile_values=[90],percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
               arcpy.management.AlterField(in_table=LTEtmpn,field="SUM",new_field_name=NewName,new_field_alias=NewName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
               if LTECount != 1:
                   arcpy.management.JoinField(in_data=TableLTEYear,in_field="OBJECTID",join_table=LTEtmpn,join_field="OBJECTID",
                        fields=NewName,fm_option="NOT_USE_FM",field_mapping=None)    
           
           arcpy.management.DeleteField(in_table=TableLTEYear,drop_field="code;ZONE_CODE;AREA;OBJECTID_1",method="DELETE_FIELDS")
           arcpy.management.AlterField(in_table=TableLTEYear,field="COUNT",new_field_name="Year",new_field_alias="YEAR",
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
           arcpy.management.CalculateField(in_table=TableLTEYear,field="Year",expression=Years[year],expression_type="PYTHON3",
                                        code_block="",field_type="DOUBLE",enforce_domains="NO_ENFORCE_DOMAINS")
        
           #end of per LTE loop 
           fields = arcpy.ListFields(TableLTEYear)  
           field_names = [field.name for field in fields]  
           with open(OutFile,'w') as f:  
               w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
               w.writerow(field_names)  
               for mow in arcpy.SearchCursor(TableLTEYear):  
                   field_vals = [mow.getValue(field.name) for field in fields]  
                   w.writerow(field_vals) 
print("Finished                              ")                            

Finished                              


In [1]:
# Climatalogical Rain and PET per month for CWD and among year changes

import arcpy
import csv
DaysRain = ["01/01", "01/02", "01/03", "01/04", "01/05", "01/06", "01/07", "01/08", "01/09", "01/10", "01/11", "01/12"]
DaysPET = ["16/01", "15/02", "16/03", "16/04", "16/05", "16/06", "16/07", "16/08", "16/09", "16/10", "16/11", "16/12"]
DaysMonth = [31, 28, 31, 30, 31, 30, 31,31,30,31,30,31]
Months = ['Jan','Feb','Mar','Apr','May', 'Jun','Jul','Aug','Sep','Oct','Nov','Dec']

FolderBase = r"C:\\DataDanny\\Agrichange\\Climate data\\"
FolderGrowthSeason = FolderBase + "Agriclimatic indicators Copernicus\\Mean of daily mean temperature\\"
FolderOutPut = FolderBase +"TemporaryOutputs\\"
FolderRain = FolderBase + "GPCC rain\\"
FolderPET = FolderBase + "PET CRU\\"

NCInfileA0 = FolderRain + "full_data_monthly_v2022_1951_1960_025.nc"
NCInfileA1 = FolderRain + "full_data_monthly_v2022_1961_1970_025.nc"
NCInfileA2 = FolderRain + "full_data_monthly_v2022_1971_1980_025.nc"
NCInfileA3 = FolderRain + "full_data_monthly_v2022_1981_1990_025.nc"
NCInfileA4 = FolderRain + "full_data_monthly_v2022_1991_2000_025.nc"
NCInfileA5 = FolderRain + "full_data_monthly_v2022_2001_2010_025.nc"
NCInfileA6 = FolderRain + "full_data_monthly_v2022_2011_2020_025.nc"

NCInfileB0 = FolderPET + "cru_ts4.07.1951.1960.pet.dat.nc"
NCInfileB1 = FolderPET + "cru_ts4.07.1961.1970.pet.dat.nc"
NCInfileB2 = FolderPET + "cru_ts4.05.1971.1980.pet.dat.nc"
NCInfileB3 = FolderPET + "cru_ts4.05.1981.1990.pet.dat.nc"
NCInfileB4 = FolderPET + "cru_ts4.05.1991.2000.pet.dat.nc"
NCInfileB5 = FolderPET + "cru_ts4.05.2001.2010.pet.dat.nc"
NCInfileB6 = FolderPET + "cru_ts4.05.2011.2020.pet.dat.nc"


VariableTypeRain = "precip"
VariableTypePET = "pet"

InputLocations = "LTE-StartSet"
Start_Year = 1955
End_YearEntered = 1960
End_Year = arcpy.sa.Plus(in_raster_or_constant1=End_YearEntered,in_raster_or_constant2= 1)

YearMonthCount = 0
with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        OutFileRain = FolderOutPut + "MonthlyRainFall" + ".csv"
        OutFilePET = FolderOutPut + "MonthlyPET" + ".csv"
        OutFileRainYear = FolderOutPut + "AnnualRainFall" + ".csv"
        OutFilePETYear = FolderOutPut + "AnnualPET" + ".csv"
        # Extract and Name Days per year
        if year < 1961:
            NCInfileRain = NCInfileA0
            NCInfilePET = NCInfileB0
        elif 1961 <= year <= 1970:    
            NCInfileRain = NCInfileA1
            NCInfilePET = NCInfileB1
        elif 1971 <= year <= 1980:
            NCInfileRain = NCInfileA2
            NCInfilePET = NCInfileB2
        elif 1981 <= year <= 1990:
            NCInfileRain = NCInfileA3
            NCInfilePET = NCInfileB3
        elif 1991 <= year <= 2000:
            NCInfileRain = NCInfileA4
            NCInfilePET = NCInfileB4
        elif 2001 <= year <= 2010:
            NCInfileRain = NCInfileA5
            NCInfilePET = NCInfileB5
        else:
            NCInfileRain = NCInfileA6
            NCInfilePET = NCInfileB6
            
        RainSumLayer = "RainSum"
        PETSumLayer = "SumPET"    
        Count = 0
        for date in range(0,len(DaysRain)):
            Message = "Running Year: " + str(year) + " "+ Months[date] + "           "
            print(Message, end="\r")
            Count = Count + 1;
            YearMonthCount = YearMonthCount + 1
            DimensionRain = "time '"+ DaysRain[date] +"/" +str(year) +"  12:00:00 AM'"  
            DimensionPET = "time '"+ DaysPET[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNametmpRain = "TmpDatesRain"
            LayerNametmpPET = "TmpDatesPET"
            arcpy.Delete_management(LayerNametmpRain) 
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfileRain, variable=VariableTypeRain, x_dimension="lon", y_dimension="lat",
                   out_raster_layer="InBetweenRain", band_dimension="", dimension_values=DimensionRain,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Times(in_raster_or_constant1="InBetweenRain",in_raster_or_constant2=1)
            out_raster.save(LayerNametmpRain)
            
            arcpy.Delete_management("InBetweenPET") 
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfilePET, variable=VariableTypePET, x_dimension="lon", y_dimension="lat",
                   out_raster_layer="InBetweenPET", band_dimension="", dimension_values=DimensionPET,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Times(in_raster_or_constant1="InBetweenPET",in_raster_or_constant2=DaysMonth[date])
            out_raster.save(LayerNametmpPET)
            
            if Count == 1:
                arcpy.Delete_management(RainSumLayer)
                arcpy.management.CopyRaster(in_raster=LayerNametmpRain,out_rasterdataset=RainSumLayer,config_keyword="",background_value=None,
                      nodata_value="",onebit_to_eightbit="NONE",colormap_to_RGB="NONE", pixel_type="",scale_pixel_value="NONE",
                      RGB_to_Colormap="NONE",format="",transform="NONE",process_as_multidimensional="CURRENT_SLICE",
                      build_multidimensional_transpose="NO_TRANSPOSE")
                arcpy.Delete_management(PETSumLayer)
                arcpy.management.CopyRaster(in_raster=LayerNametmpPET,out_rasterdataset=PETSumLayer,config_keyword="",background_value=None,
                      nodata_value="",onebit_to_eightbit="NONE",colormap_to_RGB="NONE", pixel_type="",scale_pixel_value="NONE",
                      RGB_to_Colormap="NONE",format="",transform="NONE",process_as_multidimensional="CURRENT_SLICE",
                      build_multidimensional_transpose="NO_TRANSPOSE")   
            else:
                out_raster = arcpy.sa.Plus(in_raster_or_constant1=RainSumLayer,in_raster_or_constant2= LayerNametmpRain)
                out_raster.save("InBetweenRainSum")
                arcpy.Delete_management(RainSumLayer)
                arcpy.management.CopyRaster(in_raster="InBetweenRainSum",out_rasterdataset=RainSumLayer,config_keyword="",background_value=None,
                      nodata_value="",onebit_to_eightbit="NONE",colormap_to_RGB="NONE", pixel_type="",scale_pixel_value="NONE",
                      RGB_to_Colormap="NONE",format="",transform="NONE",process_as_multidimensional="CURRENT_SLICE",
                      build_multidimensional_transpose="NO_TRANSPOSE")   
                arcpy.Delete_management("InBetweenRainSum")
                
                out_raster = arcpy.sa.Plus(in_raster_or_constant1=PETSumLayer,in_raster_or_constant2= LayerNametmpPET)
                out_raster.save("InBetweenPETSum")
                arcpy.Delete_management(PETSumLayer)
                arcpy.management.CopyRaster(in_raster="InBetweenPETSum",out_rasterdataset=PETSumLayer,config_keyword="",background_value=None,
                      nodata_value="",onebit_to_eightbit="NONE",colormap_to_RGB="NONE", pixel_type="",scale_pixel_value="NONE",
                      RGB_to_Colormap="NONE",format="",transform="NONE",process_as_multidimensional="CURRENT_SLICE",
                      build_multidimensional_transpose="NO_TRANSPOSE")   
                arcpy.Delete_management("InBetweenPETSum")
            
            TableNameFullRain = "ZonalRain"
            TableNameFullPET = "ZonalPET"
            if  YearMonthCount == 1:
                TableNametmpRain = TableNameFullRain
                TableNametmpPET = TableNameFullPET
            else:
                TableNametmpRain = "TmpRain"
                TableNametmpPET = "TmpPET"
            
            NewName = "Y_" + str(year) +"_" + Months[date]
            
            arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="code",in_value_raster= LayerNametmpRain, out_table=TableNametmpRain,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
            arcpy.management.AlterField(in_table=TableNametmpRain,field="MEAN",new_field_name=NewName,new_field_alias=NewName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
            
            arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="code",in_value_raster= LayerNametmpPET, out_table=TableNametmpPET,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
            arcpy.management.AlterField(in_table=TableNametmpPET,field="MEAN",new_field_name=NewName,new_field_alias=NewName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
            if  YearMonthCount != 1:
                arcpy.management.JoinField(in_data=TableNameFullRain,in_field="code",join_table=TableNametmpRain,join_field="code",
                    fields=NewName,fm_option="NOT_USE_FM",field_mapping=None)    
                arcpy.management.JoinField(in_data=TableNameFullPET,in_field="code",join_table=TableNametmpPET,join_field="code",
                    fields=NewName,fm_option="NOT_USE_FM",field_mapping=None)
                
        TableYearRain = "YearRain"
        TableYearPET = "YearPET"
        if  year == Start_Year:
            YeartmpRain = TableYearRain
            YeartmpPET = TableYearPET
        else:
            YeartmpRain = "TmpYearRain"
            YeartmpPET = "TmpYearPET"
        YearName = "Y_" + str(year)        
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="code",in_value_raster= RainSumLayer, out_table=YeartmpRain,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)        
        arcpy.management.AlterField(in_table=YeartmpRain,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="code",in_value_raster= PETSumLayer, out_table=YeartmpPET,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)        
        arcpy.management.AlterField(in_table=YeartmpPET,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableYearRain,in_field="code",join_table=YeartmpRain,join_field="code",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)    
            arcpy.management.JoinField(in_data=TableYearPET,in_field="code",join_table=YeartmpPET,join_field="code",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)    
        
         #Write away per year 
        for i in range(0,4):
            if i == 0:
                table = TableNameFullRain 
                OutFile = OutFileRain
            elif i == 1:
                table = TableNameFullPET
                OutFile = OutFilePET  
            elif i == 2:
                table = TableYearRain
                OutFile = OutFileRainYear 
            else:
                table = TableYearPET
                OutFile = OutFilePETYear
            fields = arcpy.ListFields(table)  
            field_names = [field.name for field in fields]  
            with open(OutFile,'w') as f:  
                w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
                w.writerow(field_names)  
                for mow in arcpy.SearchCursor(table):  
                    field_vals = [mow.getValue(field.name) for field in fields]  
                    w.writerow(field_vals)    
print("Finished                              ")                    

Finished                              


In [1]:
# Climatalogical water deficit with start at 1-01 every year

import arcpy
import csv
DaysRain = ["01/01", "01/02", "01/03", "01/04", "01/05", "01/06", "01/07", "01/08", "01/09", "01/10", "01/11", "01/12"]
DaysPET = ["16/01", "15/02", "16/03", "16/04", "16/05", "16/06", "16/07", "16/08", "16/09", "16/10", "16/11", "16/12"]
DaysMonth = [31, 28, 31, 30, 31, 30, 31,31,30,31,30,31]

FolderBase = r"C:\\DataDanny\\Agrichange\\Climate data\\"
FolderGrowthSeason = FolderBase + "Agriclimatic indicators Copernicus\\Mean of daily mean temperature\\"
FolderOutPut = FolderBase +"TemporaryOutputs\\"
FolderRain = FolderBase + "GPCC rain\\"
FolderPET = FolderBase + "PET CRU\\"

NCInfileA1 = FolderRain + "full_data_monthly_v2022_1961_1970_025.nc"
NCInfileA2 = FolderRain + "full_data_monthly_v2022_1971_1980_025.nc"
NCInfileA3 = FolderRain + "full_data_monthly_v2022_1981_1990_025.nc"
NCInfileA4 = FolderRain + "full_data_monthly_v2022_1991_2000_025.nc"
NCInfileA5 = FolderRain + "full_data_monthly_v2022_2001_2010_025.nc"
NCInfileA6 = FolderRain + "full_data_monthly_v2022_2011_2020_025.nc"

NCInfileB1 = FolderPET + "cru_ts4.07.1961.1970.pet.dat.nc"
NCInfileB2 = FolderPET + "cru_ts4.05.1971.1980.pet.dat.nc"
NCInfileB3 = FolderPET + "cru_ts4.05.1981.1990.pet.dat.nc"
NCInfileB4 = FolderPET + "cru_ts4.05.1991.2000.pet.dat.nc"
NCInfileB5 = FolderPET + "cru_ts4.05.2001.2010.pet.dat.nc"
NCInfileB6 = FolderPET + "cru_ts4.05.2011.2020.pet.dat.nc"


TypeStats = "MEAN"
NameIndicator = "CWDAnnual"
VariableTypeRain = "precip"
VariableTypePET = "pet"

InputLocations = "LTE-StartSet"
Start_Year = 2001
End_YearEntered = 2020 
End_Year = arcpy.sa.Plus(in_raster_or_constant1=End_YearEntered,in_raster_or_constant2= 1)
Previous_Month = 0 
Start_Seq = 0

YearCount = 0
with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        YearCount = YearCount + 1
        Message = "Running Year: " + str(year) + "           "
        print(Message, end="\r")
        OutFileFull = FolderOutPut + NameIndicator + "CWD" + ".csv"
        OutFileSeason = FolderOutPut + "Season" + NameIndicator + "CWD_"+ str(year) + ".csv"
        # Extract and Name Days per year
        if year < 1971:
            NCInfileRain = NCInfileA1
            NCInfilePET = NCInfileB1
        elif 1971 <= year <= 1980:
            NCInfileRain = NCInfileA2
            NCInfilePET = NCInfileB2
        elif 1981 <= year <= 1990:
            NCInfileRain = NCInfileA3
            NCInfilePET = NCInfileB3
        elif 1991 <= year <= 2000:
            NCInfileRain = NCInfileA4
            NCInfilePET = NCInfileB4
        elif 2001 <= year <= 2010:
            NCInfileRain = NCInfileA5
            NCInfilePET = NCInfileB5
        else:
            NCInfileRain = NCInfileA6
            NCInfilePET = NCInfileB6
        # Water deficit per calender year, so every year a new one with zeros is generated    
        arcpy.Delete_management("Previous_MonthDeficit") 
        out_raster = arcpy.sa.CreateConstantRaster(constant_value=0,data_type="INTEGER",cell_size=0.25,
                   extent='-180 -90 180 90 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]')
        out_raster.save("Previous_MonthDeficit")
        
        Count = 0
        for date in range(0,len(DaysRain)):
            Count = Count + 1;
            DimensionRain = "time '"+ DaysRain[date] +"/" +str(year) +"  12:00:00 AM'"  
            DimensionPET = "time '"+ DaysPET[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNametmpRain = "TmpDatesRain_" + str(Count)
            LayerNametmpPET = "TmpDatesPET_"  + str(Count)
            LayerName = "Dates_" + str(Count)
            arcpy.Delete_management(LayerNametmpRain) 
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfileRain, variable=VariableTypeRain, x_dimension="lon", y_dimension="lat",
                   out_raster_layer=LayerNametmpRain, band_dimension="", dimension_values=DimensionRain,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            arcpy.Delete_management("InBetweenPET") 
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfilePET, variable=VariableTypePET, x_dimension="lon", y_dimension="lat",
                   out_raster_layer="InBetweenPET", band_dimension="", dimension_values=DimensionPET,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Times(in_raster_or_constant1="InBetweenPET",in_raster_or_constant2=DaysMonth[date])
            out_raster.save(LayerNametmpPET)
            out_raster = arcpy.sa.Minus(in_raster_or_constant1=LayerNametmpPET,in_raster_or_constant2= LayerNametmpRain)
            out_raster.save("InBetweenCMW")
            out_raster = arcpy.sa.Plus(in_raster_or_constant1="InBetweenCMW",in_raster_or_constant2= "Previous_MonthDeficit")
            out_raster.save("InBetweenCMW2")
            arcpy.Delete_management("Previous_MonthDeficit") 
            out_raster = arcpy.sa.Con(in_conditional_raster="InBetweenCMW2",in_true_raster_or_constant=0,
                         in_false_raster_or_constant="InBetweenCMW2",where_clause="VALUE < 0")
            out_raster.save(LayerName)
            arcpy.management.CopyRaster(in_raster=LayerName,out_rasterdataset="Previous_MonthDeficit",config_keyword="",background_value=None,
                                        nodata_value="",onebit_to_eightbit="NONE",colormap_to_RGB="NONE", pixel_type="",scale_pixel_value="NONE",
                                        RGB_to_Colormap="NONE",format="",transform="NONE",process_as_multidimensional="CURRENT_SLICE",
                                        build_multidimensional_transpose="NO_TRANSPOSE")            
            if date == 0:
                LayersArrayFull = LayerName
            else:
                LayersArrayFull = LayersArrayFull + ";" + LayerName   
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFull,
                     statistics_type=TypeStats,ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save("CWDSumFull") 
        TableNameFull = "ZonalCWD"
        if  YearCount == 1:
            TableNametmp = TableNameFull
        else:
            TableNametmp = "TmpSeasonYear"
        
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="code",in_value_raster= "CWDSumFull", out_table=TableNametmp,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
        NewName = "Y_" + str(year)
        arcpy.management.AlterField(in_table=TableNametmp,field="MEAN",new_field_name=NewName,new_field_alias=NewName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR") 
        if  YearCount != 1:
            arcpy.management.JoinField(in_data=TableNameFull,in_field="code",join_table=TableNametmp,join_field="code",
                    fields=NewName,fm_option="NOT_USE_FM",field_mapping=None)                   
            
        # Extract per LTE
        InFileFirst = FolderGrowthSeason + "FirstDate_"+ str(year) + ".tif"
        InFileLast =FolderGrowthSeason + "LastDate_"+ str(year) + ".tif"
        LTECount = 0;
        arcpy.Delete_management("ZonalrastStart")     
        arcpy.Delete_management("ZonalrastLast") 
        expression=" SeqNum >= "+ str(Start_Seq)
        #expression=" SeqNum < 12"
        with arcpy.da.SearchCursor(InputLocations, ["code"], where_clause=expression) as cursor: 
            for row in cursor:          
                Message = "Running LTE: " + (row[0]) + " for " + str(year) + "           "
                print(Message, end="\r")
                exp = "code = '" + (row[0]) + "'" 
                LTECount = LTECount + 1;
                # Determine Growth season
                arcpy.analysis.Select(in_features=InputLocations,out_feature_class="TestedLTE", where_clause=exp)
                arcpy.sa.ZonalStatisticsAsTable(in_zone_data="TestedLTE", zone_field="code",in_value_raster= InFileFirst, out_table="ZonalrastStart",
                         ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                         percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
                SC = "ZonalrastStart"
                field_name = "Mean"
                values = [row[0] for row in arcpy.da.SearchCursor(SC, field_name)]
                Begintmp = values[0]
                Minusnum = arcpy.sa.Minus(in_raster_or_constant1=Begintmp,in_raster_or_constant2= 1)
                Dividenum = arcpy.sa.Divide(in_raster_or_constant1= Minusnum,in_raster_or_constant2= 3)
                Intnum = arcpy.sa.Int(in_raster_or_constant= Dividenum)
                Begin = arcpy.sa.Plus(in_raster_or_constant1=Intnum,in_raster_or_constant2= 1)
                arcpy.sa.ZonalStatisticsAsTable(in_zone_data="TestedLTE",zone_field="code", in_value_raster=InFileLast, out_table="ZonalrastLast",
                            ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                            percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360) 
                SC = "ZonalrastLast"
                field_name = "Mean"
                values = [row[0] for row in arcpy.da.SearchCursor(SC, field_name)]
                Endtmp = values[0]
                if Endtmp == 37:
                    Endtmp = 36
                Minusnum = arcpy.sa.Minus(in_raster_or_constant1=Endtmp,in_raster_or_constant2= 1)
                Dividenum = arcpy.sa.Divide(in_raster_or_constant1= Minusnum,in_raster_or_constant2= 3)
                Intnum = arcpy.sa.Int(in_raster_or_constant= Dividenum)
                End = arcpy.sa.Plus(in_raster_or_constant1=Intnum,in_raster_or_constant2= 2) # add one more being Python
                # Growth season layers to be used as array
                Count = 0;              
                for date in range(Begin,End):
                    LayerName = "Dates_" + str(date)
                    if date == Begin:
                        LayersArraySeason = LayerName
                    else:
                        LayersArraySeason = LayersArraySeason + ";" + LayerName
                # Extract from Layers 
                TableNameSeasonOut = "ZonalSeason"
                if  LTECount == 1:
                    TableNameSeason = TableNameSeasonOut
                else:
                    TableNameSeason = "TmpSeason"
                NewName = "LTE_" + row[0]
                # the cell statistics layer for full is calculated above already    
                out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArraySeason,
                             statistics_type=TypeStats,ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                             percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
                out_raster.save("MinimumYearSeason")    
                
                arcpy.sa.ZonalStatisticsAsTable(in_zone_data="TestedLTE", zone_field="code",in_value_raster= "MinimumYearSeason", out_table=TableNameSeason,
                             ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                             percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
                arcpy.management.AlterField(in_table=TableNameSeason,field="MEAN",new_field_name=NewName,new_field_alias=NewName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
                        
                if  LTECount != 1:
                    arcpy.management.JoinField(in_data="ZonalSeason",in_field="OBJECTID",join_table="TmpSeason",join_field="OBJECTID",
                         fields=NewName,fm_option="NOT_USE_FM",field_mapping=None)
        arcpy.management.DeleteField(in_table="ZonalSeason",drop_field="code;ZONE_CODE",method="DELETE_FIELDS")
        arcpy.management.AlterField(in_table="ZonalSeason",field="COUNT",new_field_name="Year",new_field_alias="YEAR",
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        arcpy.management.CalculateField(in_table="ZonalSeason",field="Year",expression=year,expression_type="PYTHON3",
                                        code_block="",field_type="DOUBLE",enforce_domains="NO_ENFORCE_DOMAINS")

         #end of per LTE loop 
        for i in range(0,2):
            if i == 0:
                table = TableNameFull 
                OutFile = OutFileFull
            else:
                table = TableNameSeasonOut 
                OutFile = OutFileSeason   
            fields = arcpy.ListFields(table)  
            field_names = [field.name for field in fields]  
            with open(OutFile,'w') as f:  
                w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
                w.writerow(field_names)  
                for mow in arcpy.SearchCursor(table):  
                    field_vals = [mow.getValue(field.name) for field in fields]  
                    w.writerow(field_vals)        

In [2]:
# GROWING SEASON

import arcpy
Days = ["05/01", "15/01", "25/01", "05/02", "15/02", "25/02", "05/03", "15/03", "25/03", "05/04", "15/04", "25/04", \
         "05/05", "15/05", "25/05", "05/06", "15/06", "25/06", "05/07", "15/07", "25/07", "05/08", "15/08", "25/08", \
         "05/09", "15/09", "25/09", "05/10", "15/10", "25/10", "05/11", "15/11", "25/11", "05/12", "15/12", "25/12"]

Folder = r"C:\\DataDanny\\Agrichange\\Climate data\\Agriclimatic indicators Copernicus\\Mean of daily mean temperature\\"
Infile = Folder + "TG_C3S-glob-agric_hadgem2-es_hist_dek_19510101-19801231_v1.1.nc"
with arcpy.EnvManager(overwriteOutput=True):
    for year in range(1955,1960):
        Count = 1
        OutFileMin = Folder + "FirstDate_" + str(year) + ".tif"
        OutFileMax = Folder + "LastDate_" + str(year) + ".tif"
        arcpy.Delete_management("Minus_Base")
        arcpy.Delete_management("Max_Base")
        arcpy.Delete_management("TG_Layer")
        for date in range(0,len(Days)):
           # f = ["TG_Layer","ExtractTmp","MinTmp"]
            #for i in f:
             #   arcpy.Delete_management(i) 
            Dimension = "time '"+ Days[date] +"/" +str(year) +"  12:00:00 AM'"  
            print(Dimension, end="\r")
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=Infile, variable="TG", x_dimension="lon", y_dimension="lat",
                out_raster_layer="TG_Layer", band_dimension="", dimension_values=Dimension,
                value_selection_method="BY_VALUE", cell_registration="CENTER")
            if date == 0:
                out_raster = arcpy.sa.Con(in_conditional_raster="TG_Layer",
                             in_true_raster_or_constant=999,
                             in_false_raster_or_constant=Count,
                             where_clause="VALUE < 278.15")
                out_raster.save("Minus_Base")
                out_raster = arcpy.sa.Con(in_conditional_raster="TG_Layer",
                             in_true_raster_or_constant=999,
                             in_false_raster_or_constant=999,
                             where_clause="VALUE < 278.15")
                out_raster.save("Max_Base")
            else:
                out_raster = arcpy.sa.Con(in_conditional_raster="TG_Layer",
                             in_true_raster_or_constant=0,
                             in_false_raster_or_constant=Count,
                             where_clause="VALUE < 278.15")
                out_raster.save("ExtractTmp")
                if date < 19:
                    out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants="Minus_Base;ExtractTmp",
                             statistics_type="MINIMUM",
                             ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                             percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
                    out_raster.save("MinTmp")
                    out_raster = arcpy.sa.Con(in_conditional_raster="MinTmp",
                                          in_true_raster_or_constant=999,
                                          in_false_raster_or_constant="MinTmp",
                                          where_clause="Value = 0")
                    out_raster.save("Minus_Base")
                else:    
                    out_raster = arcpy.sa.Con(in_conditional_raster="TG_Layer",
                             in_true_raster_or_constant=Count,
                             in_false_raster_or_constant=999,
                             where_clause="VALUE < 278.15")
                    out_raster.save("ExtractTmp")   
                    out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants="Max_Base;ExtractTmp",
                             statistics_type="MINIMUM",
                             ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                             percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
                    out_raster.save("Max_Basetmp")
                    out_raster = arcpy.sa.Con(in_conditional_raster="Max_Basetmp",
                             in_true_raster_or_constant=37,
                             in_false_raster_or_constant="Max_Basetmp",
                             where_clause="Value = 999")
                    out_raster.save("Max_Base")
            Count = Count + 1
        # Write the per year files    
        arcpy.management.CopyRaster(in_raster="Minus_Base",out_rasterdataset=OutFileMin,
                                   config_keyword="",background_value=None,nodata_value="",onebit_to_eightbit="NONE",
                                   colormap_to_RGB="NONE", pixel_type="",scale_pixel_value="NONE",RGB_to_Colormap="NONE",
                                   format="TIFF",transform="NONE",process_as_multidimensional="CURRENT_SLICE",build_multidimensional_transpose="NO_TRANSPOSE")
        arcpy.management.CopyRaster(in_raster="Max_Base",out_rasterdataset=OutFileMax,
                                config_keyword="",background_value=None,nodata_value="",onebit_to_eightbit="NONE",
                                colormap_to_RGB="NONE",pixel_type="",scale_pixel_value="NONE",RGB_to_Colormap="NONE",
                                format="TIFF",transform="NONE",process_as_multidimensional="CURRENT_SLICE",build_multidimensional_transpose="NO_TRANSPOSE")         

In [1]:
# TEMPERATURES

import arcpy
import csv
Days = ["05/01", "15/01", "25/01", "05/02", "15/02", "25/02", "05/03", "15/03", "25/03", "05/04", "15/04", "25/04", \
         "05/05", "15/05", "25/05", "05/06", "15/06", "25/06", "05/07", "15/07", "25/07", "05/08", "15/08", "25/08", \
         "05/09", "15/09", "25/09", "05/10", "15/10", "25/10", "05/11", "15/11", "25/11", "05/12", "15/12", "25/12"]
FolderBase = r"C:\\DataDanny\\Agrichange\\Climate data\\Agriclimatic indicators Copernicus\\"
FolderGrowthSeason = FolderBase + "Mean of daily mean temperature\\"
FolderOutPut = r"C:\\DataDanny\\Agrichange\\Climate data\\TemporaryOutputs\\"

#Folder = FolderBase + "Maximum value of daily maximum temperature\\"
Folder = FolderGrowthSeason
NCInfile1 = Folder + "TG_C3S-glob-agric_hadgem2-es_hist_dek_19510101-19801231_v1.1.nc"
NCInfile2 = Folder + "TG_C3S-glob-agric_hadgem2-es_hist_dek_19810101-20101231_v1.1.nc"
NCInfile3 = Folder + "TG_C3S-glob-agric_hadgem2-es_rcp8p5_dek_20110101-20401231_v1.1.nc"
TypeStats = "MEAN"
NameIndicator = "MEAN"
VariableType = "TG"

store_buffs = "LTE-StartSet"
Start_Year = 1961
End_Year = 1965 # note add 1

with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        Message = "Running Year: " + str(year) + "           "
        print(Message, end="\r")
        OutFile1 = FolderOutPut + NameIndicator + "Temp_"+ str(year) + ".csv"
        OutFile2 = FolderOutPut + "Season" + NameIndicator + "Temp_"+ str(year) + ".csv"
        # Extract and Name Days per year
        if year < 1981:
            NCInfile = NCInfile1
        elif year > 2010:
            NCInfile = NCInfile3
        else:
            NCInfile = NCInfile2
        Count = 0
        for date in range(0,len(Days)):
            Count = Count + 1;
            Dimension = "time '"+ Days[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNametmp = "TmpDates_" + str(Count)
            LayerName = "Dates_" + str(Count)
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfile, variable=VariableType, x_dimension="lon", y_dimension="lat",
            out_raster_layer=LayerNametmp, band_dimension="", dimension_values=Dimension,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Minus(in_raster_or_constant1=LayerNametmp,in_raster_or_constant2= 273.15)
            out_raster.save(LayerName) 
            if date == 0:
                LayersArrayFull = LayerName
            else:
                LayersArrayFull = LayersArrayFull + ";" + LayerName   
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFull,
                     statistics_type=TypeStats,ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save("MinimumYearFull") 
        TableName = "ZonalFull"
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=store_buffs, zone_field="code",in_value_raster= "MinimumYearFull", out_table=TableName,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
        NewName = "Y_" + str(year)
        arcpy.management.AlterField(in_table=TableName,field="MEAN",new_field_name=NewName,new_field_alias=NewName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR") 
        # Extract per LTE
        InFile1 = FolderGrowthSeason + "FirstDate_"+ str(year) + ".tif"
        InFile2 =FolderGrowthSeason + "LastDate_"+ str(year) + ".tif"
        LTECount = 0;
        arcpy.Delete_management("Zonalrast")     
        arcpy.Delete_management("Zonalrast2") 
        Start_Seq = 0
        expression=" SeqNum >= "+ str(Start_Seq)
        #expression=" SeqNum < 12"
        with arcpy.da.SearchCursor(store_buffs, ["code"], where_clause=expression) as cursor: 
            for row in cursor:          
                Message = "Running LTE: " + (row[0]) + " for " + str(year) + "           "
                print(Message, end="\r")
                exp = "code = '" + (row[0]) + "'" 
                LTECount = LTECount + 1;
                # Determine Growth season
                arcpy.analysis.Select(in_features=store_buffs,out_feature_class="TestedLTE", where_clause=exp)
                arcpy.sa.ZonalStatisticsAsTable(in_zone_data="TestedLTE", zone_field="code",in_value_raster= InFile1, out_table="Zonalrast",
                         ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                         percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
                SC = "Zonalrast"
                field_name = "Mean"
                values = [row[0] for row in arcpy.da.SearchCursor(SC, field_name)]
                Begintmp = values[0]
                Begin = arcpy.sa.Int(in_raster_or_constant= Begintmp)
                arcpy.sa.ZonalStatisticsAsTable(in_zone_data="TestedLTE",zone_field="code", in_value_raster=InFile2, out_table="Zonalrast2",
                            ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                            percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360) 
                SC = "Zonalrast2"
                field_name = "Mean"
                values = [row[0] for row in arcpy.da.SearchCursor(SC, field_name)]
                Endtmp = values[0]
                End = arcpy.sa.Int(in_raster_or_constant= Endtmp)
                # Growth season layers to be used as array
                Count = 0;              
                for date in range(Begin,End):
                    LayerName = "Dates_" + str(date)
                    if date == Begin:
                        LayersArraySeason = LayerName
                    else:
                        LayersArraySeason = LayersArraySeason + ";" + LayerName
                # Extract from Layers        
                if  LTECount == 1:
                    TableNameSeason = "ZonalSeason"
                else:
                    TableNameSeason = "TmpSeason"
                NewName = "LTE_" + row[0]
                # the cell statistics layer for full is calculated above already    
                out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArraySeason,
                             statistics_type=TypeStats,ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                             percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
                out_raster.save("MinimumYearSeason")    
                
                arcpy.sa.ZonalStatisticsAsTable(in_zone_data="TestedLTE", zone_field="code",in_value_raster= "MinimumYearSeason", out_table=TableNameSeason,
                             ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                             percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
                arcpy.management.AlterField(in_table=TableNameSeason,field="MEAN",new_field_name=NewName,new_field_alias=NewName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
                        
                if  LTECount != 1:
                    arcpy.management.JoinField(in_data="ZonalSeason",in_field="OBJECTID",join_table="TmpSeason",join_field="OBJECTID",
                         fields=NewName,fm_option="NOT_USE_FM",field_mapping=None)
        arcpy.management.DeleteField(in_table="ZonalSeason",drop_field="code;ZONE_CODE",method="DELETE_FIELDS")
        arcpy.management.AlterField(in_table="ZonalSeason",field="COUNT",new_field_name="Year",new_field_alias="YEAR",
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        arcpy.management.CalculateField(in_table="ZonalSeason",field="Year",expression=year,expression_type="PYTHON3",
                                        code_block="",field_type="DOUBLE",enforce_domains="NO_ENFORCE_DOMAINS")

        #end of per LTE loop 
        for i in range(0,2):
            if i == 0:
                table = "ZonalFull" 
                OutFile = OutFile1 
            else:
                table = "ZonalSeason" 
                OutFile = OutFile2    
            fields = arcpy.ListFields(table)  
            field_names = [field.name for field in fields]  
            with open(OutFile,'w') as f:  
                w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
                w.writerow(field_names)  
                for mow in arcpy.SearchCursor(table):  
                    field_vals = [mow.getValue(field.name) for field in fields]  
                    w.writerow(field_vals)                     

In [4]:
import arcpy
import csv
Folder = r"C:\\DataDanny\\Agrichange\\Climate data\\Agriclimatic indicators Copernicus\\Mean of daily mean temperature\\"
with arcpy.EnvManager(overwriteOutput=True):
    for year in range(1965,2023):
        InFile1 = Folder + "FirstDate_"+ str(year) + ".tif"
        InFile2 = Folder + "LastDate_"+ str(year) + ".tif"
        OutFile = Folder + "GrowingSeason_"+ str(year) + ".csv"
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data="LTE_Start",zone_field="code", in_value_raster=InFile1, out_table="Zonalrast",
                 ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                 percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data="LTE_Start",zone_field="code", in_value_raster=InFile2, out_table="Zonalrast2",
                 ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                 percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)        
        arcpy.management.AlterField(in_table="Zonalrast",field="MEAN",new_field_name="GS_Start",new_field_alias="GS_Start",
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        arcpy.management.AlterField(in_table="Zonalrast2",field="MEAN",new_field_name="GD_End",new_field_alias="GD_End",
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        arcpy.management.JoinField(in_data="Zonalrast",in_field="code",join_table="Zonalrast2",join_field="code",
                         fields="GD_End",fm_option="NOT_USE_FM",field_mapping=None)
        arcpy.management.AddField(in_table="Zonalrast",field_name="Year",field_type="DOUBLE",field_precision=10,
                         field_scale=5,field_length=None,field_alias="",field_is_nullable="NULLABLE",
                         field_is_required="NON_REQUIRED",field_domain="")
        arcpy.management.CalculateField(in_table="Zonalrast",field="Year",expression=year,expression_type="PYTHON3",
                         code_block="",field_type="TEXT",enforce_domains="NO_ENFORCE_DOMAINS")
        table = "Zonalrast" 
        fields = arcpy.ListFields(table)  
        field_names = [field.name for field in fields]  
        with open(OutFile,'w') as f:  
            w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
            w.writerow(field_names)  
            for mow in arcpy.SearchCursor(table):  
                field_vals = [mow.getValue(field.name) for field in fields]  
                w.writerow(field_vals)

In [41]:
import arcpy
Days = ["05/01", "15/01", "25/01", "05/02", "15/02", "25/02", "05/03", "15/03", "25/03", "05/04", "15/04", "25/04", \
         "05/05", "15/05", "25/05", "05/06", "15/06", "25/06", "05/07", "15/07", "25/07", "05/08", "15/08", "25/08", \
         "05/09", "15/09", "25/09", "05/10", "15/10", "25/10", "05/11", "15/11", "25/11", "05/12", "15/12", "25/12"]

Folder = r"C:\\DataDanny\\Agrichange\\Climate data\\Agriclimatic indicators Copernicus\\Minimum value of the daily minimum Temperature\\"
Infile = Folder + "TNn_C3S-glob-agric_hadgem2-es_hist_dek_19810101-20101231_v1.1.nc"
with arcpy.EnvManager(overwriteOutput=True):
    for year in range(1981,2011):
        Count = 1
        arcpy.Delete_management("Minus_Base")
        arcpy.Delete_management("Max_Base")
        arcpy.Delete_management("TG_Layer")
        for date in range(0,len(Days)):
            Dimension = "time '"+ Days[date] +"/" +str(year) +"  12:00:00 AM'"  
            print(Dimension, end="\r")
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=Infile, variable="TNn", x_dimension="lon", y_dimension="lat",
                out_raster_layer="TG_Layer", band_dimension="", dimension_values=Dimension,
                value_selection_method="BY_VALUE", cell_registration="CENTER")
            arcpy.management.CopyRaster(in_raster="TG_Layer",out_rasterdataset=OutFile,
                                config_keyword="",background_value=None,nodata_value="",onebit_to_eightbit="NONE",
                                colormap_to_RGB="NONE",pixel_type="",scale_pixel_value="NONE",RGB_to_Colormap="NONE",
                                format="TIFF",transform="NONE",process_as_multidimensional="CURRENT_SLICE",build_multidimensional_transpose="NO_TRANSPOSE")         